In [194]:
# coding:utf-8
"""
本篇为单独调试get_coupon()函数的调试记录，后期将加入UserMainData
"""
import pandas as pd
import numpy as np
from impala.dbapi import connect
from impala.util import as_pandas
import cPickle

def easy_print(terms,tname, records=10): #逐条描述前五条记录，方便仔细查看表中值的特征
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()     
    
    sql = 'SELECT ' + terms + ' FROM ' + tname + ' LIMIT 5' 
    cur.execute(sql)
    read_in = as_pandas(cur)    
    cur.close()
    conn.close()
    
    #print in a format
    columns = read_in.columns.values
    values = read_in.values
    records = []
    for v in values:
        records.append(zip(columns, v))
    
    for ix,r in enumerate(records):
        print('----------第%s条记录--------' % (ix+1))
        for k, v in r:
            print('%s: %s' % (k, v))
            
def full_print(terms,tname): #打出全表所选列，以方便进行统计查看规律，决定处理方法
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()     
    
    sql = 'SELECT ' + terms + ' FROM ' + tname
    cur.execute(sql)
    df = as_pandas(cur)
    print df
    cur.close()
    conn.close()

def get_coupon():
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()     
    # 1. 对用户获取的红包数量和总金额进行统计，最后为'customerid','coupon_get_cnt','coupon_get_sum'
    sql = 'SELECT customer_id as customerid,coupon_value,red_envelop_id FROM km_tbl_get_coupon_records'
    cur.execute(sql)
    df = as_pandas(cur)
    # 统计用户参与的活动数
    group_by_used = df.groupby('customerid',as_index=False).count()
    #统计用户获取过的红包金额
    group_by_money = df.groupby('customerid',as_index=False).sum()
    # 按照customerid合并两部分数据
    got = pd.merge(group_by_used,group_by_money,left_on='customerid',right_on='customerid')
    # 清洗数据（改列名和drop不要的列）
    got.drop('coupon_value_x',axis=1,inplace=True)
    got.columns = ['customerid','coupon_get_cnt','coupon_get_sum']

    # 2. 对每个用户最多红包获取的来源商家进行统计，最后为'customerid','source_company_id','source_company_cnt'
    sql = 'SELECT customer_id as customerid,source_company_id FROM km_tbl_get_coupon_records'
    cur.execute(sql)
    df = as_pandas(cur)
    df['source_company_cnt'] = 1
    company = df.groupby(['customerid','source_company_id'],as_index=False).count()
    # 只保留每个用户id对应的获取次数最多的商家次数
    company = company.sort_values(['customerid','source_company_cnt'],ascending=[1,0])
    # drop_duplicates默认保留第一条
    company.drop_duplicates(subset='customerid',inplace=True)
    
    # 3. 对用户红包获取过的红包类型数进行统计,最后为'customerid','get_coupon_type_cnt'
    sql = 'SELECT customer_id as customerid,coupon_id FROM km_tbl_get_coupon_records'
    cur.execute(sql)
    df = as_pandas(cur)
    df['cnt'] = 1
    df = df.groupby(['customerid','coupon_id'],as_index=False).count()
    type_cnt = df.groupby('customerid',as_index=False).count()
    type_cnt.drop('cnt',axis=1,inplace=True)
    type_cnt.columns = ['customerid','get_coupon_type_cnt']
    
    # 4. 对以上几个表进行merge
    df = pd.merge(got,company,left_on='customerid',right_on='customerid')
    df = pd.merge(df,type_cnt,left_on='customerid',right_on='customerid')
    
    # 5. 和来源商家数据联系在一起
    sql = """SELECT companyid as source_company_id,roomnum as get_com_roomnum,regioncode as get_com_regioncode,
    managetype as get_com_managetype,vodsystemtype as get_com_vodsystemtype FROM kdw_tbl_companybase"""
    cur.execute(sql)
    src_company = as_pandas(cur)    
    df = pd.merge(df,src_company,how='outer',left_on='source_company_id',right_on='source_company_id')
    
    # 6.判断用户是否在不同商家获得红包
    df['get_coupon_crosscom'] = map(int,df['coupon_get_cnt'] > df['source_company_cnt'])
    return df
    cur.close()
    conn.close()    

In [195]:
df = get_coupon()

## 6. 判断用户是否在不同商家获得红包

In [191]:
df.head()

,customerid,coupon_get_cnt,coupon_get_sum,source_company_id,source_company_cnt,get_coupon_type_cnt,get_com_roomnum,get_com_regioncode,get_com_managetype,get_com_vodsystemtype,get_coupon_crosscom
0,1003913,12,215,4617,9,4,63,500000,1,0,1
1,1536143,1,5,4617,1,1,63,500000,1,0,0
2,3303677,2,20,4617,2,2,63,500000,1,0,0
3,3931247,4,35,4617,4,4,63,500000,1,0,0
4,3937144,1,20,4617,1,1,63,500000,1,0,0


In [192]:
df.shape

(145904, 11)

## 5. 和来源商家数据merge

In [174]:
df.head()

,customerid,coupon_get_cnt,coupon_get_sum,source_company_id,source_company_cnt,get_coupon_type_cnt,get_com_roomnum,get_com_regioncode,get_com_managetype,get_com_vodsystemtype
0,1003913,12,215,4617,9,4,63,500000,1,0
1,1536143,1,5,4617,1,1,63,500000,1,0
2,3303677,2,20,4617,2,2,63,500000,1,0
3,3931247,4,35,4617,4,4,63,500000,1,0
4,3937144,1,20,4617,1,1,63,500000,1,0


In [175]:
df.shape

(145904, 10)

## 4. 对以上几个表进行merge

### 连接got和company结果

In [161]:
df.head()

,customerid,coupon_get_cnt,coupon_get_sum,source_company_id,source_company_cnt
0,1003913,12,215,4617,9
1,1004944,2,10,8407,2
2,1006609,6,115,4270,5
3,1006661,3,35,4270,2
4,100910,2,15,6491,2


### 将上表和type_cnt进行merge

In [164]:
df.head()

,customerid,coupon_get_cnt,coupon_get_sum,source_company_id,source_company_cnt,get_coupon_type_cnt
0,1003913,12,215,4617,9,4
1,1004944,2,10,8407,2,1
2,1006609,6,115,4270,5,4
3,1006661,3,35,4270,2,3
4,100910,2,15,6491,2,2


In [167]:
df.shape

(145083, 6)

## 3. 对用户红包获取的红包类型数统计结果

In [140]:
df

,customerid,get_coupon_type_cnt
0,1003913,4
1,1004944,1
2,1006609,4
3,1006661,3
4,100910,2
5,1009386,1
6,1009656,1
7,101273,1
8,1014444,1
9,1016017,4


## 查看source_type_name有没有区分度：完全没有，放弃此属性

In [112]:
pd.value_counts(df['source_type_name'])

电视端二维码    339357
Name: source_type_name, dtype: int64

## 2. 对用户获取红包来源商家统计

In [95]:
df

,customerid,source_company_id,cnt
0,1003913,-1,3
1,1003913,4617,9
2,1004944,8407,2
3,1006609,-1,1
4,1006609,4270,5
5,1006661,-1,1
6,1006661,4270,2
7,100910,6491,2
8,1009386,3170,1
9,1009656,-1,1


### 以customerid升序和source_company_cnt降序排列结果

In [106]:
df

,customerid,source_company_id,source_company_cnt
1,1003913,4617,9
0,1003913,-1,3
2,1004944,8407,2
4,1006609,4270,5
3,1006609,-1,1
6,1006661,4270,2
5,1006661,-1,1
7,100910,6491,2
8,1009386,3170,1
9,1009656,-1,1


### 去重只保留每个customerid第一条的结果

In [109]:
df

,customerid,source_company_id,source_company_cnt
1,1003913,4617,9
2,1004944,8407,2
4,1006609,4270,5
6,1006661,4270,2
7,100910,6491,2
8,1009386,3170,1
9,1009656,-1,1
10,101273,-1,1
11,1014444,-1,1
12,1016017,-1,3


## 1. 对用户获取的红包数量和总金额进行统计

In [76]:
df

,customerid,coupon_get_cnt,coupon_get_sum
0,1003913,12,215
1,1004944,2,10
2,1006609,6,115
3,1006661,3,35
4,100910,2,15
5,1009386,1,5
6,1009656,1,20
7,101273,1,10
8,1014444,1,15
9,1016017,5,85


## 抽出表中customerid,coupon_value,red_envelop_id属性进行的一些统计尝试

In [12]:
df.groupby(['customerid']).sum()

,coupon_value
customerid,
1003913,215
1004944,10
1006609,115
1006661,35
100910,15
1009386,5
1009656,20
101273,10
1014444,15


In [15]:
df1 = df.groupby(['customerid','red_envelop_id']).sum()
df1

coupon_value
customerid red_envelop_id              
1003913    171                       80
           95                       135
1004944    169                       10
1006609    169                       75
           171                       30
           93                        10
1006661    169                        5
           171                       20
           93                        10
100910     169                       15
1009386    91                         5
1009656    171                       20
101273     129                       10
1014444    173                       15
1016017    169                       20
           171                       65
1017391    169                       40
           41                        15
           93                        10
1018310    169                        5
           41                        20
1019803    171                      500
           95                        15
1020619    171                       10
1022811    173                       20
           91                         5
1026056    95                        15
1026952    173                       20
           93                        10
1030733    173                       20
...                                 ...
951300     93                        10
954077     169                       20
954828     169                       30
957011     91                         5
           93                        10
957080     131                        5
           169                       30
           173                       20
958861     169                        5
959288     169                       20
961556     137                       30
           169                        5
969283     169                       10
970335     173                       10
973677     171                       45
978610     91                         5
978795     169                       10
           171                       55
           41                        20
           91                         5
           93                        30
           95                        15
983841     91                         5
           93                        40
984698     169                       30
           91                         5
           93                        20
987874     169                       30
           91                         5
997143     169                       10

[231106 rows x 1 columns]

In [16]:
df2 = df.groupby(['customerid','red_envelop_id']).count()
df2

coupon_value
customerid red_envelop_id              
1003913    171                        3
           95                         9
1004944    169                        2
1006609    169                        4
           171                        1
           93                         1
1006661    169                        1
           171                        1
           93                         1
100910     169                        2
1009386    91                         1
1009656    171                        1
101273     129                        1
1014444    173                        1
1016017    169                        2
           171                        3
1017391    169                        2
           41                         1
           93                         1
1018310    169                        1
           41                         1
1019803    171                       26
           95                         1
1020619    171                        1
1022811    173                        1
           91                         1
1026056    95                         1
1026952    173                        1
           93                         1
1030733    173                        1
...                                 ...
951300     93                         1
954077     169                        2
954828     169                        1
957011     91                         1
           93                         1
957080     131                        1
           169                        1
           173                        1
958861     169                        1
959288     169                        2
961556     137                        2
           169                        1
969283     169                        1
970335     173                        1
973677     171                        3
978610     91                         1
978795     169                        1
           171                        3
           41                         1
           91                         1
           93                         3
           95                         1
983841     91                         1
           93                         4
984698     169                        1
           91                         1
           93                         2
987874     169                        3
           91                         1
997143     169                        1

[231106 rows x 1 columns]

In [19]:
df1['get_coupon_cnt'] = df2['coupon_value']
df1

coupon_value  get_coupon_cnt
customerid red_envelop_id                              
1003913    171                       80               3
           95                       135               9
1004944    169                       10               2
1006609    169                       75               4
           171                       30               1
           93                        10               1
1006661    169                        5               1
           171                       20               1
           93                        10               1
100910     169                       15               2
1009386    91                         5               1
1009656    171                       20               1
101273     129                       10               1
1014444    173                       15               1
1016017    169                       20               2
           171                       65               3
1017391    169                       40               2
           41                        15               1
           93                        10               1
1018310    169                        5               1
           41                        20               1
1019803    171                      500              26
           95                        15               1
1020619    171                       10               1
1022811    173                       20               1
           91                         5               1
1026056    95                        15               1
1026952    173                       20               1
           93                        10               1
1030733    173                       20               1
...                                 ...             ...
951300     93                        10               1
954077     169                       20               2
954828     169                       30               1
957011     91                         5               1
           93                        10               1
957080     131                        5               1
           169                       30               1
           173                       20               1
958861     169                        5               1
959288     169                       20               2
961556     137                       30               2
           169                        5               1
969283     169                       10               1
970335     173                       10               1
973677     171                       45               3
978610     91                         5               1
978795     169                       10               1
           171                       55               3
           41                        20               1
           91                         5               1
           93                        30               3
           95                        15               1
983841     91                         5               1
           93                        40               4
984698     169                       30               1
           91                         5               1
           93                        20               2
987874     169                       30               3
           91                         5               1
997143     169                       10               1

[231106 rows x 2 columns]